# M/EEG exam instructions
- Assignment format: 
    - mandatory: a notebook with your answers 
    - optional: an additional document with your answers to the conceptual questions
- Please send your assignment to M.Corsi's email address
- Deadline: 
    - **March, 20th, 8AM.** Please note that an **extension will not be proposed.**

In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.colors import TwoSlopeNorm

import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.stats import permutation_cluster_1samp_test as pcluster_test
from mne.time_frequency import tfr_multitaper


## Part 1 - Connectivity and Networks

Here is an EEG dataset to load:

In [2]:
 
#Define the parameters
subject = 1  # use data from subject 1
runs = [6, 10, 14]  # Motor imagery: hands vs feet

# Extract raw data
fnames = eegbci.load_data(subject=subject, runs=runs)
raw = concatenate_raws([read_raw_edf(f, preload=True) for f in fnames])
raw.rename_channels(lambda x: x.strip("."))  # remove dots from channel names
events, _ = mne.events_from_annotations(raw, event_id=dict(T1=2, T2=3))
channel_names = raw.info['ch_names']

# Extract trials between -1s and 4s
tmin, tmax = -1, 4
event_ids = dict(hands=2, feet=3)  # map event IDs to tasks
epochs = mne.Epochs(
    raw,
    events,
    event_ids,
    tmin - 0.5,
    tmax + 0.5,
    baseline=None,
    preload=True,
)


Extracting EDF parameters from /Users/marieconstance.corsi/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/marieconstance.corsi/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/marieconstance.corsi/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T1', 'T2']
Not setting metadata
45 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 45 events and 961 origin

In [3]:
from mne_connectivity import spectral_connectivity_epochs
from mne_connectivity.viz import plot_connectivity_circle

hands = epochs['hands']
feet = epochs['feet']

fmin, fmax = 8, 13 # mu band
conhands = spectral_connectivity_epochs(hands, method='imcoh', fmin=fmin, fmax=fmax, faverage=True) # compute ImCoh of Mu-band averaged over epochs

con_matrix_hands = conhands.get_data("dense") # Get the data

# plot
fig, axs = plt.subplots(figsize=(14, 5))
im = axs.imshow(conhands.get_data("dense"), vmin=0, vmax=1)
fig.colorbar(im, ax=axs)
plt.show()


Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 2016 connections
    using t=-1.500s..4.500s for estimation (961 points)
    frequencies: 8.2Hz..13.0Hz (30 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: Imaginary Coherence
    computing cross-spectral density for epoch 1
    computing cross-spectral density for epoch 2
    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10
    computing cross-spectral density for epoch 11
    computing cross-spectr

2025-02-11 14:29:26.633 python[19960:497918] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-11 14:29:26.633 python[19960:497918] +[IMKInputSession subclass]: chose IMKInputSession_Modern


## Questions:
- For each condition:
    - Compute and plot the connectivity matrices based on the estimation of the imaginary coherence averaged over the mu band and across the epochs. What does it say about potential changes between the tasks performed by the subject?
    - Compute and plot the associated node strength averaged across the epochs. What does it say about potential changes between the tasks performed by the subject?

- Here is the plot of the statistical difference between MI and Rest conditions obtained from imaginary coherence (left) and the results obtained with the node strength (right): What do you observe? Is it neurophysiologically meaningful?
![Figure_ImCoh](./MI_Rest_ImCoh.png)


## Part 2 - Features in BCI

Here is an EEG dataset to load:

In [4]:
#Define the parameters
subject = 1  # use data from subject 1
runs = [6, 10, 14]  # Motor imagery: hands vs feet

# Extract raw data
fnames = eegbci.load_data(subject=subject, runs=runs)
raw = concatenate_raws([read_raw_edf(f, preload=True) for f in fnames])
raw.rename_channels(lambda x: x.strip("."))  # remove dots from channel names
events, _ = mne.events_from_annotations(raw, event_id=dict(T1=2, T2=3))

# Extract trials between -1s and 4s
channelsOfInterest = "T7", "C3", 'O1' # to get the full list of channels you can type: raw.info['ch_names']
tmin, tmax = -1, 4
event_ids = dict(hands=2, feet=3)  # map event IDs to tasks
epochs = mne.Epochs(
    raw,
    events,
    event_ids,
    tmin - 0.5,
    tmax + 0.5,
    picks=(channelsOfInterest),
    baseline=None,
    preload=True,
)

# Compare power spectra computed in each condition/channel 
freqs = np.arange(2, 45)
vmin, vmax = -1, 1.5  # set min and max ERDS values in plot
baseline = (-1, 0)  # baseline interval (in s)
cnorm = TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)  # min, center & max ERDS

kwargs = dict(
    n_permutations=100, step_down_p=0.05, seed=1, buffer_size=None, out_type="mask"
)  # for cluster test

# Time-Frequency decomposition
tfr = tfr_multitaper(
    epochs,
    freqs=freqs,
    n_cycles=freqs,
    use_fft=True,
    return_itc=False,
    average=False,
    decim=2,
)
tfr.crop(tmin, tmax).apply_baseline(baseline, mode="percent")

nb_channels = len(channelsOfInterest)
for event in event_ids:
    # select desired epochs for visualization
    tfr_ev = tfr[event]
    fig, axes = plt.subplots(
        1, 4, figsize=(12, 4), gridspec_kw={"width_ratios": [10, 10, 10, 1]}
    )
    for ch, ax in enumerate(axes[:-1]):  # for each channel
        # positive clusters
        _, c1, p1, _ = pcluster_test(tfr_ev.data[:, ch], tail=1, **kwargs)
        # negative clusters
        _, c2, p2, _ = pcluster_test(tfr_ev.data[:, ch], tail=-1, **kwargs)

        # note that we keep clusters with p <= 0.05 from the combined clusters
        # of two independent tests; in this example, we do not correct for
        # these two comparisons
        c = np.stack(c1 + c2, axis=2)  # combined clusters
        p = np.concatenate((p1, p2))  # combined p-values
        mask = c[..., p <= 0.05].any(axis=-1)

        # plot TFR (ERDS map with masking)
        tfr_ev.average().plot(
            [ch],
            cmap="RdBu",
            cnorm=cnorm,
            axes=ax,
            colorbar=False,
            show=False,
            mask=mask,
            mask_style="mask",
        )

        ax.set_title(epochs.ch_names[ch], fontsize=10)
        ax.axvline(0, linewidth=1, color="black", linestyle=":")  # event
        if ch != 0:
            ax.set_ylabel("")
            ax.set_yticklabels("")
    fig.colorbar(axes[0].images[-1], cax=axes[-1]).ax.set_yscale("linear")
    fig.suptitle(f"ERDS ({event})")
    plt.show()

Extracting EDF parameters from /Users/marieconstance.corsi/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/marieconstance.corsi/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /Users/marieconstance.corsi/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T1', 'T2']
Not setting metadata
45 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 45 events and 961 origin

  0%|          | Permuting : 0/99 [00:00<?,       ?it/s]

Step-down-in-jumps iteration #1 found 1 cluster to exclude from subsequent iterations


  0%|          | Permuting : 0/99 [00:00<?,       ?it/s]

Step-down-in-jumps iteration #2 found 0 additional clusters to exclude from subsequent iterations
Using a threshold of -1.724718
stat_fun(H1): min=-4.1380237268582585 max=3.447987864084401
Running initial clustering …
Found 98 clusters


...

Step-down-in-jumps iteration #1 found 1 cluster to exclude from subsequent iterations


  0%|          | Permuting : 0/99 [00:00<?,       ?it/s]

Step-down-in-jumps iteration #2 found 0 additional clusters to exclude from subsequent iterations
Using a threshold of -1.713872
stat_fun(H1): min=-3.6160177594410863 max=3.5139046340567477
Running initial clustering …
Found 60 clusters


  0%|          | Permuting : 0/99 [00:00<?,       ?it/s]

Step-down-in-jumps iteration #1 found 0 clusters to exclude from subsequent iterations
No baseline correction applied


## Questions: 
- Please describe the observations you can make from the maps. Are there neurophysiologically relevant/meaningful?
- To what extent such observations are informative of BCI performance?
- If you were the experimenter, based on the previous observations, which (electrode(s); frequency bin(s)) couples would you pick to extract the features? Why?

## Part 3 - Machine Learning & BCI

## Context

Here is a publicly available [BCI dataset](http://moabb.neurotechx.com/docs/generated/moabb.datasets.BNCI2014_001.html#moabb.datasets.BNCI2014_001) (cf below to load the data and to get information regarding the experimental information). In the following lines of code we defined two classification pipelines (CSP+LDA: Common Spatial Patterns + LDA, RG+LR:Riemannian Geometry + Logistic Regression), and we plotted their performances from a dataset composed of 2 subjects.

## Questions
- What observations can be made from the plots? 
- Instead of those implemented here, what framework would you propose to extract, select, and classify the features? Why? How would you assess the performance of your approach?

## BONUS
- Implement below your framework and compare it to the other pipelines (namely RG+LR and CSP+LDA). 
- What are your conclusions? Do you have some suggestion(s) to improve the performance of your framework?

In [5]:
import warnings

from mne.decoding import CSP
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

import moabb
from moabb.datasets import BNCI2014001 # note: if you use MOABB>1.1, please change it as BNCI2014_001
from moabb.evaluations import WithinSessionEvaluation
from moabb.paradigms import LeftRightImagery

moabb.set_log_level("info")
mne.set_log_level("CRITICAL")
warnings.filterwarnings("ignore")


###### PIPELINES TO BE COMPARED (do not modify it!) ######
# baseline pipeline to be used to make the comparison, please complete the following line with your framework
pipelines = {}
pipelines["CSP+LDA"] = make_pipeline(CSP(n_components=8), LDA())
pipelines["RG+LR"] = make_pipeline(
    Covariances(), TangentSpace(), LogisticRegression(solver="lbfgs")
)
### BONUS - implementation of your framework ######
#pipelines["MyPipeline"] =


###### DATASET TO BE USED (do not modify it!) - downloading it the first time can take some time ######
dataset = BNCI2014001() # if you are using a more recent version of moabb please change it as BNCI2014_001
subj = [1, 2]
dataset.subject_list = subj


###### DEFINITION OF THE PARADIGM & EVALUATION (do not modify it!) ######
paradigm = LeftRightImagery()
evaluation = WithinSessionEvaluation(
    paradigm=paradigm, datasets=dataset, overwrite=False
)
results = evaluation.process(pipelines)
#print(results.head()) # if you want to look at it...


2025-02-11 14:31:26,941 WARNING MainThread moabb.utils BNCI2014001 has been renamed to BNCI2014_001. BNCI2014001 will be removed in version 1.1.
2025-02-11 14:31:26,941 WARNING MainThread moabb.datasets.base The dataset class name 'BNCI2014001' must be an abbreviation of its code 'BNCI2014-001'. See moabb.datasets.base.is_abbrev for more information.
2025-02-11 14:31:26,942 INFO MainThread moabb.evaluations.base Processing dataset: BNCI2014-001
BNCI2014-001-WithinSession:   0%|                                                                                                           | 0/2 [00:00<?, ?it/s]


In [6]:
###### SCRIPT TO PLOT THE RESULTS (do not modify it unless you want to add your framework!) #########

# Plot the global distribution of the performance
fig, axes = plt.subplots(1, 2, figsize=[8, 4], sharey=True)

sns.stripplot(
    data=results,
    y="score",
    x="pipeline",
    ax=axes[0],
    jitter=True,
    alpha=0.5,
    zorder=1,
    palette="rocket",
)
sns.pointplot(data=results, y="score", x="pipeline", ax=axes[0], palette="rocket")

axes[0].set_ylabel("ROC AUC")
axes[0].set_ylim(0.5, 1)

paired = results.pivot_table(
    values="score", columns="pipeline", index=["subject", "session"]
)
paired = paired.reset_index()

sns.regplot(data=paired, y="RG+LR", x="CSP+LDA", ax=axes[1], fit_reg=False)
axes[1].plot([0, 1], [0, 1], ls="--", c="k")
axes[1].set_xlim(0.5, 1)

plt.show()

# Plot the individual distribution of the performance
g = sns.catplot(
    kind="bar",
    x="score",
    y="subject",
    hue="pipeline",
    col="dataset",
    height=12,
    aspect=0.5,
    data=results,
    orient="h",
    palette="rocket",
)
plt.show()

## Part 4 - Experimental considerations

You plan to launch a new protocol based on EEG acquisitions:
- What are the two main types of artifacts you may observe? Please indicate one example for each of them.
- What are the main steps that compose an EEG processing pipeline?

Now you are conducting and experimental protocol in BCI. It consists in 5 sessions of right hand motor imagery vs rest. After the fourth session training sessions subject Y still shows a global performance of 60%. 
At each session:
- You instructed the subject to perform a right motor imagery when the visual target was up and to remain at rest when the visual target was down.
- You always selected the same features (power spectra in CP3 at 10Hz & 14Hz, and in C3 at 12Hz & 16Hz).

Based on these elements, what would be your suggestions to help the subject Y improving their performance at session 5?